In [3]:
# Warning control
import warnings
import os

warnings.filterwarnings("ignore")

from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## Agent


In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match " "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
)

## CREW AI tools

- Read any directory
- read any file
- search the internet


In [7]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

directory_read_tool = DirectoryReadTool(
    directory="./instructions"
)  ## just read one single directory
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

## TOOL

- for simplicity purpose we are making sure its just retuen answer as positive


In [9]:
from crewai_tools import BaseTool


class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = (
        "Analyzes the sentiment of text "
        "to ensure positive and engaging communication."
    )

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## TASK


In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [13]:
## this task is to make sure we are talking to customer in a positive manner and do more research if any about the compmany that 1st agent didn;t do
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

# CREW


In [14]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2,
    memory=True,
)

## output


In [15]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch",
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To start building a comprehensive profile on DeepLearningAI, I should first gather general information about the company, their recent milestones, and any key personnel that could be important in our interactions. This will allow me to understand their potential needs and how our solutions could be aligned with their business.

Action: Search the internet
Action Input: {"search_query": "DeepLearn

In [16]:
from IPython.display import Markdown

Markdown(result)

Subject: Elevating AI Education Together

Dear Andrew,

I hope this message finds you in great spirits. Having followed DeepLearningAI's journey under your visionary leadership, I'm truly impressed by your recent strides in AI education, particularly the launch of cutting-edge courses on generative AI technologies. Your commitment to making world-class AI education accessible globally resonates deeply with us at CrewAI.

At CrewAI, we specialize in developing advanced AI tools that can significantly enhance educational content and delivery. Given your recent initiatives, I believe there's a unique opportunity for us to collaborate. Integrating our AI solutions could provide DeepLearningAI students with more interactive and immersive learning experiences, setting a new benchmark in AI education.

I would be thrilled to discuss how we can tailor our tools to best fit your educational goals and perhaps co-develop exclusive courses that highlight practical AI applications. Our aim is to support DeepLearningAI in nurturing the next generation of AI talent.

Looking forward to the possibility of working together to create something truly impactful for the AI community.

Warmest regards,

[Your Name]
Lead Sales Representative, CrewAI
[Your Contact Information]